# Part 3: Weakly supervised part-of-speech tagging

In this part, we will work on a different type of tasks, which is called sequence labeling. Instead of having one label for an entire text, in sequence labeling, we assign a label to each token in the text.
Specifically we chose Part-of-speech (POS) tagging, which concerns the task of assigning a POS tag that indicates a grammatical type, to a word based on its definition and context.

In order to perform weakly supervised POS tagging, we will employ the [skweak toolkit](https://github.com/NorskRegnesentral/skweak).
We will create labeling functions to assign POS tags based on syntactic analysis and grammatical rules.


In [32]:
# Imports
%load_ext autoreload
%autoreload 2

import re
import os

import pandas as pd

import spacy
from spacy.tokens import Span
from spacy.training import Corpus

import skweak

from scripts.skweak_ner_eval import evaluate
from scripts.utils import load_data_split, get_frequent_words, tag_all

pd.set_option('display.max_rows', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

We will use the [English corpus](https://universaldependencies.org/treebanks/en_ewt/index.html) from Universal Dependencies, a framework that contains consistent grammatical annotations across many different languages.
The texts in the corpus come from five types of web media: weblogs, newsgroups, emails, reviews, and Yahoo! answers and consist of 254,825 words and 16,621 sentences.

Skweak operates on spaCy ``doc`` objects, so the dataset is loaded in this format.

In [3]:


# Path to the dataset file
data_path = os.path.join("corpus", "UD_English-EWT")

# Create a blank spacy pipeline
nlp = spacy.blank("xx")

# Load training data
reader = Corpus(os.path.join(data_path, "train.spacy"))
train_data = list(reader(nlp))

# Load test data
reader = Corpus(os.path.join(data_path, "test.spacy"))
test_data = list(reader(nlp))

In [4]:
# Get the doc objects
train_docs = [doc.reference.copy() for doc in train_data]
print("There are", len(train_docs), "documents in the training set")
test_docs = [doc.reference.copy() for doc in test_data]
print("There are", len(test_docs), "documents in the test set")

There are 12543 documents in the training set
There are 2077 documents in the test set


## Part-of-speech (POS) tagging

The goal is to assign a POS tag to each token.

For this tutorial, we will use the following subset of the [universal POS tags](https://universaldependencies.org/u/pos/index.html):
1. **DET**: determiner, which is a word that modifies nouns or noun phrases and expresses the reference of the noun phrase in context.
2. **NUM**: numeral. It is a word that expresses a number and a relation to the number, such as quantity, sequence, frequency or fraction.
3. **PROPN**: proper noun is a noun that is the name of a specific individual, place, or object.
4. **ADJ**: adjective, which is a word that typically modifies nouns and specifies their properties or attributes.
5. **NOUN**: noun, which is a part of speech typically denoting a person, place, thing, animal or idea.

In [120]:
all_labels = ["DET",  "PROPN", "ADJ", "NOUN"]

In [6]:
# Function that assigns the gold labels based on the subset we chose
def assign_gold_labels(docs):
    for doc in docs:
        # print([s.text for s in doc.sents])
        ents = []
        tok_pos = []
        for tok in doc:
            if tok.pos_ in all_labels:
                # print(tok.pos_)
                tok_pos.append(tok.pos_)
                ents.append(Span(doc, tok.i, tok.i + 1, tok.pos_))
            else:
                tok_pos.append("O")
        doc.set_ents(ents)
        # print(tok_pos)

In [6]:
assign_gold_labels(train_docs)
assign_gold_labels(test_docs)

## 3.1 Labeling functions

In the first step, we find the 200 most frequent words in our training corpus and use a lexicon to label these words. In the second step, we mannually annotate the 50 most frequent words.
Finally, for each POS tag we will create the following labeling functions: 

*   DET --> Lexicon with determiners.
*   NUM --> If the token is a number.
*   PROPN --> A word that is capitalized.
*   ADJ --> Suffixes: “able”, “al”, “ful”, “ic”, “ive”, “less”, “ous”, ”y”, “ish”, “ible”, "est".
*   NOUN --> 1. Suffixes: "ment", "tion", "sion", "xion", "ant", "ent", "ee", "er", "or", "ism", "ist", "ness", "ship", "ity", "ance", "ence", "ar", "or", "y", "acy", "age" , 2. Linguistic rule: if the previous word is a DET, a NUM or an ADJ, then the current one is a NOUN.
*   VERB --> 1. Suffixes: "ing", "ate", "en", "ed", "ify", "ise", "ize", 2. Linguistic rule: if the previous word is a NOUN, then the current one is a VERB, 3. Previous word is a form of "be".

#### Lexicon LF

In [18]:
common_words = get_frequent_words(train_docs, 200)
print(common_words[:5])

["'s", "n't", 'would', 'one', 'like']


In [19]:
# Load the lexicon
with open("noun_vb_adj_list.txt") as f:
    lines = f.readlines()

# Create a dictionary with the words and their pos tags
lexicon = {}
for l in lines:
    values = l.replace("\n", "").split("\t")
    lexicon[values[0]] = values[1]

In [20]:
print("There are", len(lexicon), "words in the lexicon.")
print(list(lexicon.items())[:5])

There are 3387 words in the lexicon.
[('people', 'NOUN'), ('history', 'NOUN'), ('way', 'NOUN'), ('art', 'NOUN'), ('world', 'NOUN')]


In [21]:
# How many of the common words we found exist in the lexicon
len((list(set(common_words) & set(list(lexicon.keys())))))

121

In [16]:
# Lexicon LF
def common_word_detector(doc):
    for token in doc:
        # If the frequent word exists in the lexicon use its assigned pos tag
        if token.text.lower() in common_words and token.text.lower() in list(lexicon.keys()):
            yield token.i, token.i + 1, lexicon[token.text.lower()]


word_lf = skweak.heuristics.FunctionAnnotator("common_words", common_word_detector)


#### Manual annotation LF

In [17]:
# Manual annotation
top50_words = get_frequent_words(train_docs, 50)
print(top50_words)

["'s", "n't", 'would', 'one', 'like', 'time', 'get', 'know', 'also', 'us', 'good', 'could', 'new', 'go', 'please', '$', 'people', 'may', 'back', 'said', 'even', 'work', 'bush', 'well', 'want', 'great', 'way', 'see', 'best', 'place', 'take', "'m", 'going', 'service', 'need', 'thanks', 'make', 'many', 'year', 'number', 'day', 'two', 'think', 'much', 'food', 'let', 'first', 'call', '2', 'help']


In [22]:
# Annotate the words that belong to our chosen subset
manual_tags = {
    "one": "NUM",
    "like": "VERB",
    "time": "NOUN",
    "get": "VERB",
    "know": "VERB",
    "good": "ADJ",
    "could": "VERB",
    "new": "ADJ",
    "go": "VERB",
    "please": "VERB",
    "people": "NOUN",
    "said": "VERB",
    "work": "VERB",
    "bush": "NOUN",
    "want": "VERB",
    "great": "ADJ",
    "way": "NOUN",
    "see": "VERB",
    "best": "ADJ",
    "place": "NOUN",
    "take": "VERB",
    "going": "VERB",
    "service": "NOUN",
    "need": "VERB",
    "make": "VERB",
    "year": "NOUN",
    "number": "NOUN",
    "day": "NOUN",
    "two": "NUM",
    "think": "VERB",
    "food": "NOUN",
    "let": "VERB",
    "first": "ADJ",
    "call": "VERB",
    "2": "NUM",
    "help": "VERB"
}

In [23]:
# Manual POS tags LF
def manual_pos_tagger(doc):
    for token in doc:
        if token.text.lower() in manual_tags:
            yield token.i, token.i + 1, manual_tags[token.text.lower()]


manual_pos_lf = skweak.heuristics.FunctionAnnotator("manual_pos", manual_pos_tagger)


#### DET LF

In [24]:
# Use a lexicon of determiners
tries = skweak.gazetteers.extract_json_data("det.json")
det_lf = skweak.gazetteers.GazetteerAnnotator("determiners", tries, case_sensitive=False)


Extracting data from det.json
Populating trie for class DET (number: 47)


#### NUM LF

In [25]:
# Use a regular expression pattern to look for digits
def num_detector(doc):
    for token in doc:
        if re.search("\d+", token.text):
            yield token.i, token.i + 1, "NUM"


num_lf = skweak.heuristics.FunctionAnnotator("numerals", num_detector)


#### PROPN LF

In [26]:
# Check if the fist letter of a word is capitalized
def propn_detector(doc):
    for token in doc:
        if token.i == 0:
            # For the first word of a sentence, check if all letters are capitalized
            if token.text.isupper():
                yield token.i, token.i + 1, "PROPN"
        else:
            if token.text.isupper() or token.text[0].isupper():
                yield token.i, token.i + 1, "PROPN"


propn_lf = skweak.heuristics.FunctionAnnotator("proper_nouns", propn_detector)


#### ADJ LFs

In [27]:
# Look for common suffixes and prefixes
def adj_detector_suffixes(doc):
    suffixes = ("able", "al", "ful", "ic", "ive", "less", "ous", "y", "ish", "ible", "ent", "est")
    for token in doc:
        if len(token.text) > 3 and token.text.endswith(suffixes):
            yield token.i, token.i + 1, "ADJ"


# Look for common prefixes
def adj_detector_prefixes(doc):
    prefixes = ("un", "im", "in", "ir", "il", "non", "dis")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "ADJ"


# If the previous word is a form of "be" and the current word does not end with "ing" and was not labeled as DET, then it's an adjective
def adj_detector(doc):
    weak_labels = ["O"] * len(doc)
    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            prev = doc[token.i - 1].text.lower()
            if prev in ["be", "been", "being", "am", "is", "are", "was", "were"] and (
                    not token.text.endswith("ing")) and weak_labels[token.i] == "O":
                yield token.i, token.i + 1, "ADJ"


# If the previous word is labeld as DET or NUM, then the current word is an adjective
def adj_detector_ling(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "ADJ"


adj_lf1 = skweak.heuristics.FunctionAnnotator("adjectives1", adj_detector_suffixes)
adj_lf2 = skweak.heuristics.FunctionAnnotator("adjectives2", adj_detector_prefixes)
adj_lf3 = skweak.heuristics.FunctionAnnotator("adjectives3", adj_detector)
adj_lf4 = skweak.heuristics.FunctionAnnotator("adjectives4", adj_detector_ling)


#### NOUN LF

Let's create a labeling function that looks for common noun suffixes. Can you think of some?

In [28]:
# ***********************************
def noun_detector_suffixes(doc):
    suffixes = ("ment", "tion", "sion", "xion", "ant", "ent", "ee", "er", "or",
                "ism", "ist", "ness", "ship", "ity", "ance", "ence",
                "ar", "or", "y", "acy", "age")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().endswith(suffixes):
            yield token.i, token.i + 1, "NOUN"

# ***********************************

In [29]:
# Look for common prefixes
def noun_detector_prefixes(doc):
    prefixes = (
        "anti", "auto", "bi", "co", "counter", "dis", "ex", "hyper", "in", "inter", "kilo", "mal", "mega", "mis",
        "mini", "mono", "neo", "out", "poly", "pseudo", "re", "semi", "sub", "super", "sur", "tele", "tri", "ultra",
        "under", "vice")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "NOUN"


# # If the previous word is labeld as DET, NUM or ADJ, then the current word is an noun
def noun_detector_ling(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives3"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives4"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "NOUN"


noun_lf1 = skweak.heuristics.FunctionAnnotator("nouns1", noun_detector_suffixes)
noun_lf2 = skweak.heuristics.FunctionAnnotator("nouns2", noun_detector_prefixes)
noun_lf3 = skweak.heuristics.FunctionAnnotator("nouns3", noun_detector_ling)


## Apply LFs

In [132]:
# Put all LFs in a list
lfs = [
    word_lf, manual_pos_lf,
    det_lf, num_lf, propn_lf,
    adj_lf1, adj_lf2, adj_lf3, adj_lf4,
    noun_lf1, noun_lf2, noun_lf3
]

train_docs = load_data_split("train", all_labels)
train_docs = tag_all(train_docs, lfs)

In [133]:
# Print some of the assigned weak labels
for doc in train_docs[0:3]:
    skweak.utils.display_entities(doc, ["determiners", "nouns1"])

In [134]:
# Train HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm = hmm.fit(train_docs, n_iter=8)

Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 2


         1     -481878.0229             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 3


         2     -454753.3161      +27124.7068


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 4


...

         6     -426856.2936       +2859.8395


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents
Starting iteration 8


         7     -423013.7484       +3842.5452


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Number of processed documents: 5000
Number of processed documents: 6000
Number of processed documents: 7000
Number of processed documents: 8000
Number of processed documents: 9000
Number of processed documents: 10000
Number of processed documents: 11000
Number of processed documents: 12000
Finished E-step with 12543 documents


         8     -417588.7557       +5424.9927


In [135]:
# Majority voting
mv = skweak.aggregation.MajorityVoter("mv", all_labels)

In [136]:

# Apply LFs, HMM and MV to the test docs
test_docs = load_data_split("test", all_labels)
test_docs = tag_all(test_docs, lfs + [mv, hmm])

## Evaluate

In [137]:
df = evaluate(test_docs, all_labels, [
    # "common_words", "manual_pos",
    # "determiners", "numerals", "proper_nouns",
    # "adjectives1", "adjectives2", "adjectives3", "adjectives4",
    # "nouns1", "nouns2", "nouns3", 
    "mv", "hmm"
])

In [138]:
df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      17.3 %     hmm            0.183       0.323   0.234
                    mv             0.367       0.389   0.378
DET      19.4 %     hmm            0.687       0.912   0.784
                    mv             0.708       0.845   0.770
NOUN     42.2 %     hmm            0.299       0.176   0.222
                    mv             0.346       0.386   0.364
PROPN    21.2 %     hmm            0.623       0.279   0.386
                    mv             0.576       0.482   0.524
macro               hmm            0.448       0.422   0.434
                    mv             0.499       0.526   0.512
micro               hmm            0.404       0.366   0.384
                    mv             0.467       0.496   0.482
weighted            hmm            0.423       0.366   0.392
                    mv             0.468       0.496   0.482

#### Analysis:

* We see that POS tags like determiners and numerals are easier to detect and we can achieve a very good F1 score with just one simple LF.
* Other POS tags like adjectives and nouns, which rely more on the context are harder to detect and require more complicated rules.
* For adjectives the LF that uses suffixes works the best, while the syntactic rules are less accurate. On the contrary, for nouns the LF that is based on syntactic analysis has the best results. For both POS tags, the LFs that use prefixes do not yield good results.
* Despite its simplicity, majority voting outperforms HMM on most of the POS tags and overall achieves a higher macro F1 score.

## Run LFs on a smaller subset

What if we use a smaller amount of training data? How will that affect the performance?

In [139]:
subset_train_docs = load_data_split("train", all_labels, 500)

#### Annotate the most common 50 words

In [140]:
# Manual annotation
top50_words = get_frequent_words(subset_train_docs, 50)
print(top50_words)

["'s", 'bush', 'al', 'india', 'would', 'iraq', 'us', 'iraqi', "n't", 'one', 'many', 'even', 'indian', 'said', 'new', 'war', 'musharraf', 'peace', 'years', 'country', 'military', 'israel', 'two', 'also', 'national', 'time', 'chernobyl', 'pakistan', 'government', 'kashmir', 'sri', 'elections', 'know', 'qaeda', 'may', 'president', 'power', 'last', 'another', 'lanka', 'posada', 'back', 'could', 'state', 'general', 'made', 'much', 'party', 'united', 'people']


In [141]:
manual_tags = {
    "bush": "NOUN",
    "al": "PROPN",
    "india": "PROPN",
    "iraq": "PROPN",
    "iraqi": "ADJ",
    "indian": "ADJ",
    "said": "VERB",
    "new": "ADJ",
    "war": "NOUN",
    "musharraf": "PROPN",
    "peace": "NOUN",
    "years": "NOUN",
    "country": "NOUN",
    "military": "NOUN",
    "israel": "PROPN",
    "two": "NUM",
    "national": "ADJ",
    "time": "NOUN",
    "chernobyl": "PROPN",
    "pakistan": "PROPN",
    "government": "NOUN",
    "kashmir": "PROPN",
    "sri": "PROPN",
    "elections": "NOUN",
    "know": "VERB",
    "qaeda": "PROPN",
    "president": "NOUN",
    "power": "NOUN",
    "last": "NOUN",
    "another": "ADJ",
    "lanka": "PROPN",
    "posada": "PROPN",
    "could": "VERB",
    "general": "ADJ",
    "made": "VERB",
    "party": "NOUN",
    "united": "VERB",
    "people": "NOUN",
}

In [142]:
# ***********************************

# Apply LFs to the subset docs
for doc in subset_train_docs:
    for lf in lfs:
        doc = lf(doc)

In [143]:
# Train HMM
hmm_2 = skweak.aggregation.HMM("hmm_2", all_labels, redundancy_factor=1.0)
hmm_2 = hmm_2.fit(subset_train_docs, n_iter=8)

Starting iteration 1
Finished E-step with 500 documents
Starting iteration 2


         1      -33070.3558             +nan


Finished E-step with 500 documents
Starting iteration 3


         2      -31391.1937       +1679.1620


Finished E-step with 500 documents
Starting iteration 4


...

         6      -28811.0492        +476.4665


Finished E-step with 500 documents
Starting iteration 8


         7      -28586.8415        +224.2078


Finished E-step with 500 documents


         8      -28536.3379         +50.5036


In [144]:
# Majority voting
mv = skweak.aggregation.MajorityVoter("mv", all_labels)


In [145]:
# Apply LFs, HMM and MV to the test docs

test_docs_2 = load_data_split("test", all_labels)

for doc in test_docs_2:
    for lf in  lfs + [mv, hmm_2]:
        doc = lf(doc)

In [146]:
# Evaluate
subset_df = evaluate(test_docs_2, all_labels, [
    # "common_words", "manual_pos",
    # "determiners", "numerals", "proper_nouns",
    # "adjectives1", "adjectives2", "adjectives3", "adjectives4",
    # "nouns1", "nouns2", "nouns3", 
    "mv", "hmm_2"
])

In [147]:
subset_df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      17.3 %     hmm_2          0.185       0.331   0.238
                    mv             0.367       0.389   0.378
DET      19.4 %     hmm_2          0.688       0.963   0.802
                    mv             0.708       0.845   0.770
NOUN     42.2 %     hmm_2          0.321       0.173   0.224
                    mv             0.346       0.386   0.364
PROPN    21.2 %     hmm_2          0.600       0.581   0.590
                    mv             0.576       0.482   0.524
macro               hmm_2          0.448       0.512   0.478
                    mv             0.499       0.526   0.512
micro               hmm_2          0.435       0.439   0.436
                    mv             0.467       0.496   0.482
weighted            hmm_2          0.428       0.440   0.434
                    mv             0.468       0.496   0.482

In [148]:
df

tok_precision  tok_recall  tok_f1
label    proportion model                                   
ADJ      17.3 %     hmm            0.183       0.323   0.234
                    mv             0.367       0.389   0.378
DET      19.4 %     hmm            0.687       0.912   0.784
                    mv             0.708       0.845   0.770
NOUN     42.2 %     hmm            0.299       0.176   0.222
                    mv             0.346       0.386   0.364
PROPN    21.2 %     hmm            0.623       0.279   0.386
                    mv             0.576       0.482   0.524
macro               hmm            0.448       0.422   0.434
                    mv             0.499       0.526   0.512
micro               hmm            0.404       0.366   0.384
                    mv             0.467       0.496   0.482
weighted            hmm            0.423       0.366   0.392
                    mv             0.468       0.496   0.482